# Make Web Site

In [1]:
import sys, os, io, datetime
import json
import random
import requests
import zipfile
import glob
import pandas as pd
from slugify import slugify
from tqdm.notebook import tqdm
from html import escape
from collections import defaultdict

In [2]:
import cdli

In [3]:
pd.set_option("display.max_columns", None)

In [4]:
supported_langs = set(["akk", "elx", "sux"])

In [5]:
wwwroot = os.path.abspath("../dist")
os.makedirs(wwwroot, exist_ok=True)
wwwroot

'/Users/fak/Dropbox/Projects/CuneiformTranslators/dist'

## Download the Catalog

In [6]:
cat = cdli.get_catalog()

/Users/fak/Dropbox/Projects/CuneiformTranslators/tools/cdli.py:97: DtypeWarning: Columns (0,1,2,3,4,6,8,9,10,11,13,14,16,17,19,22,23,24,25,26,27,28,29,31,32,33,37,39,42,45,46,49,52,53,54,55,56,57,58,59,60,62,63) have mixed types. Specify dtype option on import or set low_memory=False.
  cat = pd.read_csv(io.StringIO(cat_csv))


In [7]:
cat

,accession_no,accounting_period,acquisition_history,alternative_years,ark_number,atf_source,atf_up,author,author_remarks,cdli_collation,cdli_comments,citation,collection,composite_id,condition_description,date_entered,date_of_origin,date_remarks,date_updated,dates_referenced,db_source,designation,dumb,dumb2,electronic_publication,elevation,excavation_no,external_id,findspot_remarks,findspot_square,genre,google_earth_collection,google_earth_provenience,height,id,id_text2,id_text,join_information,language,lineart_up,material,museum_no,object_preservation,object_type,period,period_remarks,photo_up,primary_publication,provenience,provenience_remarks,publication_date,publication_history,published_collation,seal_id,seal_information,stratigraphic_level,subgenre,subgenre_remarks,surface_preservation,text_remarks,thickness,translation_source,width,object_remarks
0,NaN,NaN,NaN,NaN,21198/zz001q0dtm,"Englund, Robert K.",NaN,CDLI,"31x61x18; Lú A 14-16.30-32.48-50; M XVIII, auf...",NaN,NaN,NaN,"Vorderasiatisches Museum, Berlin, Germany",Q000002,NaN,12/4/2001,00.00.00.00,NaN,2020-03-14,00.00.00.00,20011204 protocuneiform_catalogue,"CDLI Lexical 000002, ex. 065",NaN,NaN,NaN,NaN,"W 06435,a",NaN,auf Hügeloberfläche in der Nähe des Südbaues,"M XVIII,?",Lexical,NaN,NaN,31,1,0,1,NaN,undetermined,150ppi 20160630,clay,VAT 01533,NaN,tablet,Uruk III (ca. 3200-3000 BC),NaN,NaN,"CDLI Lexical 000002, ex. 065",Uruk (mod. Warka),NaN,2015ff.,"Englund, Robert K. & Nissen, Hans J., ATU 3 (1...",NaN,NaN,NaN,NaN,Archaic Lu2 A (witness),NaN,NaN,NaN,18,no translation,61,NaN
1,NaN,NaN,NaN,NaN,21198/zz001q0dv4,"Englund, Robert K.",NaN,CDLI,30x48x13; Lú A 13-15.23-25.?; Fundstelle wie W...,NaN,NaN,NaN,"Vorderasiatisches Museum, Berlin, Germany",Q000002,NaN,12/4/2001,00.00.00.00,NaN,2018-10-20,00.00.00.00,20011204 protocuneiform_catalogue,"CDLI Lexical 000002, ex. 066",NaN,NaN,NaN,NaN,"W 06435,b",NaN,auf der Hügeloberfläche in der Nähe des Südbaues,"M XVIII,?",Lexical,NaN,NaN,30,2,0,2,NaN,undetermined,150ppi 20160630,clay,VAT 15263,NaN,tablet,Uruk III (ca. 3200-3000 BC),NaN,NaN,"CDLI Lexical 000002, ex. 066",Uruk (mod. Warka),NaN,2015ff.,"Englund, Robert K. & Nissen, Hans J., ATU 3 (1...",NaN,NaN,NaN,NaN,Archaic Lu2 A (witness),NaN,NaN,NaN,13,no translation,48,NaN
2,NaN,NaN,NaN,NaN,21198/zz001q0dwn,"Englund, Robert K.",NaN,"Englund, Robert K. & Nissen, Hans J.","42x53x19; Vocabulary 9; Qa XVI,2, unter der Ab...",NaN,NaN,NaN,"Vorderasiatisches Museum, Berlin, Germany",NaN,NaN,12/4/2001,NaN,NaN,2020-01-26,NaN,20011204 protocuneiform_catalogue,"ATU 3, pl. 081, W 9123,d",NaN,NaN,NaN,NaN,"W 09123,d",NaN,"unter der Abgleichung der Schicht III, 1,5 m ü...","Qa XVI,2",Lexical,NaN,NaN,42,3,0,3,NaN,undetermined,150ppi 20160630,clay,VAT 15253,NaN,tablet,Uruk IV (ca. 3350-3200 BC),NaN,NaN,"ATU 3, pl. 081, W 9123,d",Uruk (mod. Warka),NaN,1993,"ATU 1, 539",NaN,NaN,NaN,NaN,Archaic Vocabulary (witness),Text category: 15-09; Foreign ID: LVO 9,NaN,NaN,19,no translation,53,NaN
3,NaN,NaN,NaN,NaN,21198/zz001q0dx5,"Englund, Robert K.",NaN,CDLI,26x23x23; Lú A 9-10.?.?; Fundstelle wie W 9123...,NaN,NaN,NaN,"Vorderasiatisches Museum, Berlin, Germany",Q000002,NaN,12/4/2001,00.00.00.00,NaN,2018-10-20,00.00.00.00,20011204 protocuneiform_catalogue,"CDLI Lexical 000002, ex. 051",NaN,NaN,NaN,NaN,"W 09169,d",NaN,"unter der Abgleichung der Schicht III, 1,5 m ü...","Qa XVI,2",Lexical,NaN,NaN,26,4,0,4,NaN,undetermined,150ppi 20160630,clay,VAT 15168,NaN,tablet,Uruk IV (ca. 3350-3200 BC),NaN,NaN,"CDLI Lexical 000002, ex. 051",Uruk (mod. Warka),NaN,2015ff.,"Englund, Robert K. & Nissen, Hans J., ATU 3 (1...",NaN,NaN,NaN,NaN,Archaic Lu2 A (witness),NaN,NaN,NaN,23,no translation,23,NaN
4,NaN,NaN,NaN,NaN,21198/zz001q0dzp,"Englund, Robert K.",NaN,CDLI,"29x36x20; Lú A Vorläufer; Qa XVI,2, unter der ...",NaN,NaN,NaN,"Vorderasiatisches Museum, Berlin, Germany",Q000002,NaN,12/4/2001,00.00.00.00,NaN,2018-10-20,00.00.00.00,20011204 protocuneiform_catalogue,"CDLI Lexical 000002, ex. 172",NaN,NaN,NaN,NaN,"W 09206,k",

In [8]:
len(cat)

352771

In [9]:
cat.columns

Index(['accession_no', 'accounting_period', 'acquisition_history',
       'alternative_years', 'ark_number', 'atf_source', 'atf_up', 'author',
       'author_remarks', 'cdli_collation', 'cdli_comments', 'citation',
       'collection', 'composite_id', 'condition_description', 'date_entered',
       'date_of_origin', 'date_remarks', 'date_updated', 'dates_referenced',
       'db_source', 'designation', 'dumb', 'dumb2', 'electronic_publication',
       'elevation', 'excavation_no', 'external_id', 'findspot_remarks',
       'findspot_square', 'genre', 'google_earth_collection',
       'google_earth_provenience', 'height', 'id', 'id_text2', 'id_text',
       'join_information', 'language', 'lineart_up', 'material', 'museum_no',
       'object_preservation', 'object_type', 'period', 'period_remarks',
       'photo_up', 'primary_publication', 'provenience', 'provenience_remarks',
       'publication_date', 'publication_history', 'published_collation',
       'seal_id', 'seal_information', 's

https://cdli.ucla.edu/search/archival_view.php?ObjectID=P256681

In [10]:
cat[cat["id_text"]==256681]

,accession_no,accounting_period,acquisition_history,alternative_years,ark_number,atf_source,atf_up,author,author_remarks,cdli_collation,cdli_comments,citation,collection,composite_id,condition_description,date_entered,date_of_origin,date_remarks,date_updated,dates_referenced,db_source,designation,dumb,dumb2,electronic_publication,elevation,excavation_no,external_id,findspot_remarks,findspot_square,genre,google_earth_collection,google_earth_provenience,height,id,id_text2,id_text,join_information,language,lineart_up,material,museum_no,object_preservation,object_type,period,period_remarks,photo_up,primary_publication,provenience,provenience_remarks,publication_date,publication_history,published_collation,seal_id,seal_information,stratigraphic_level,subgenre,subgenre_remarks,surface_preservation,text_remarks,thickness,translation_source,width,object_remarks
106190,NaN,NaN,NaN,NaN,21198/zz001s1m1w,"de Ridder, Alba",NaN,"Stol, Marten",NaN,NaN,NaN,NaN,University of Pennsylvania Museum of Archaeolo...,NaN,NaN,2/24/2005,NaN,NaN,2021-06-12,NaN,20050224 fitzgerald_upenn,"AbB 11, 029",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Letter,NaN,NaN,?,106192,0,256681,NaN,Akkadian,NaN,clay,UM 29-16-076,NaN,tablet,Old Babylonian (ca. 1900-1600 BC),NaN,600ppi 20160630,"AbB 11, 029",Nippur (mod. Nuffar),NaN,1986,NaN,NaN,NaN,NaN,NaN,NaN,Letter; 7x8x1 line,NaN,NaN,?,no translation,?,NaN


## Get the Human Transliterations

In [11]:
all_publications = cdli.get_atf()

Parsing atf


In [12]:
print("Merging transliterations with catalog")
merged_pubs = cdli.merge_atf_with_catalog(all_publications, cat, tqdm)

Merging transliterations with catalog


  0%|          | 0/134560 [00:00<?, ?it/s]

## Get the ML Translations

In [13]:
translations_json_path = "../data/ml_translations.json"
translations = json.loads(str(open(translations_json_path, "rb").read(), "utf8"))
translations = translations["akk_to_en"]
len(translations)

380004

In [14]:
num_publications = len(cat)
num_transliterations = len(merged_pubs)
num_translations = len([x for x in merged_pubs if x.has_translations()])
print("    num_publications:", num_publications)
print("num_transliterations:", num_transliterations)
print("    num_translations:", num_translations)

    num_publications: 352771
num_transliterations: 134545
    num_translations: 5437


In [28]:
output_pubs = [p for p in merged_pubs if p.language in supported_langs]
len(output_pubs)

121188

## Find Publications we have translations for

In [29]:
tgt_lang = "en"

translated_pubs = []

for pub in tqdm(output_pubs):
    has_new_translations = False
    has_ml_translations = True
    for a in pub.text_areas:
        for l in a.lines:
            s = l.text
            if len(s) > 1:
                has_new_translations = has_new_translations or (tgt_lang not in l.languages)
                if s in translations:
                    l.languages["ml_"+tgt_lang] = translations[s]
                else:
                    has_ml_translations = False
    pub.has_new_translations = has_new_translations
    pub.has_ml_translations = has_ml_translations
    if has_ml_translations:
        translated_pubs.append(pub)
    
newly_translated_pubs = [x for x in translated_pubs if x.has_new_translations]
print(len(translated_pubs), "translated_pubs")
print(len(newly_translated_pubs), "newly_translated_pubs")

  0%|          | 0/121188 [00:00<?, ?it/s]

22696 translated_pubs
20547 newly_translated_pubs


In [30]:
newly_translated_pubs[0]

Publication(5984, 'akk', [TextArea('tablet', []), TextArea('obverse', []), TextArea('column 1', [TextLine("1'.", '[n] 4(bur3@c) _GAN2_', {'ml_en': '... 4 bur3 field'}), TextLine("2'.", 'E2#? HA? GU4? x', {'ml_en': '..'}), TextLine("3'.", 'in ur-sa6{ki}', {'ml_en': 'in Ursa'}), TextLine("4'.", '6(bur3@c) _GAN2_', {'ml_en': '6 bur3 field'}), TextLine("5'.", 'x x x', {'ml_en': '..'}), TextLine("6'.", 'x _GAN2#_ [...]', {'ml_en': '... field'}), TextLine("7'.", '[...]', {'ml_en': '..'}), TextLine("8'.", '[...]', {'ml_en': '..'}), TextLine("9'.", '2(bur3@c) _GAN2_', {'ml_en': '2 bur3 field'}), TextLine("10'.", '_GAN2 sa10_', {'ml_en': 'the field(s) purchased'}), TextLine("11'.", 'asz2-te4', {'ml_en': 'from'}), TextLine("12'.", 'inim-ma-ni#-zi#', {'ml_en': 'Inimanizi'})]), TextArea('column 2', [TextLine('1.', 'en-na-il', {'ml_en': 'Enna-il'}), TextLine('2.', '_lugal_ kisz', {'ml_en': 'king of the world'}), TextLine('3.', '_alan#?_-[su?]', {'ml_en': '(this) statue(?)'}), TextLine('4.', '[...]'

## Data Dimensions

In [31]:
browser_dimensions = [
    ("new", lambda p: ["new" if p.has_new_translations else "old"]),
    ("language", lambda p: [p.language]),
    ("object_type", lambda p: [cdli.get_object_type(p.object_type)]),
    ("genre", lambda p: cdli.get_genres(p.genre)),
]

## HTML Components

## HTML Pages

In [32]:
def get_file_path(site_path):
    return f"{wwwroot}{site_path}"

def get_page_file_path(site_path):
    return f"{get_file_path(site_path)}.html"
    

In [33]:
def header(title, f):
    f.write(f"<html>\n<head>\n<title>{escape(title)}</title>\n<meta name='viewport' content='width=device-width, initial-scale=1'>\n<style>{style}</style></head>\n")
    f.write(f"<body>\n<div class='content'><h1>{escape(title)}</h1>\n")
    f.write(f"<p class='otitle'>Translated by <a href=\"https://huggingface.co/praeclarum/cuneiform\">praeclarum/cuneiform</a></p>\n")
    
def footer(f):
    f.write(f"</div></body>\n</html>")

In [34]:
def start_page(path, title):
    file_path = get_page_file_path(path)
    file_dir = os.path.dirname(file_path)
    os.makedirs(file_dir, exist_ok=True)
    f = open(file_path, "wt")
    print(f"Writing {path} at {file_path}")
    header(title, f)
    return f

def end_page(f):
    footer(f)

In [35]:
def output_browser(path, pubs, ignore_dim_value, f):
    next_pages = []
    if len(pubs) == 0:
        return next_pages
    f.write(f"<section>\n")
    for dname, dselect in browser_dimensions:
        vgroups = defaultdict(lambda: [])
        for p in pubs:
            for v in dselect(p):
                if v == ignore_dim_value:
                    continue
                vgroups[v].append(p)
        if len(vgroups) < 2:
            continue
        f.write(f"<h1>{escape(dname)}</h1>\n")    
        for gv in vgroups.keys():
            gpubs = vgroups[gv]
            if len(gpubs) > 0:
                next_pages.append((gv, gpubs))
                f.write(f"<a href='{path}/{gv}/'>{len(gpubs)} {escape(gv)}</a>\n")            
    f.write(f"</section>\n")
    f.write(f"<section>\n")
    if len(pubs) < 100 or len(next_pages) == 0:
        for p in pubs:
            output_pub(p, f)        
    else:
        f.write(f"<p>{len(pubs)} publications. Narrow the list by choosing links above.</p>\n")
    f.write(f"</section>\n")
    return next_pages

In [36]:
style = """
body { font-family: sans-serif; }
div.content { margin: 0 auto; max-width: 512px; }
h1 { text-align: center; }
p.src { font-size: 80%; font-style: italic;}
.otitle { text-align: center; }
h1.otitle { margin-top: 1em; padding-top: 1em; border-top:solid 2px rgba(128,128,128,0.5); }
.langid {
    font-size: 80%; font-weight: bold; 
    background-color: rgba(128,128,128,0.25); border-radius:0.2em;
    padding: 0 0; min-width: 5em; display: inline-block;
    text-align: center; margin-right: 1em;
}
.lang-akkts, .lang-elxts, .lang-suxts, .lang-akk, .lang-elx, .lang-sux {
    font-style: italic; font-size: 80%;
    opacity: 0.75;
    }
.lang-en { opacity: 0.5; font-size: 80%; }
.lang-ml_en {}
"""

In [37]:
language_sort = {
    "akk": 0,
    "akkts": 1,
    "elx": 2,
    "elxts": 3,
    "sux": 4,
    "suxts": 5,
    "ml_en": 100,
    "en": 1000,
    "fr": 1001,
}

def output_pub(p, f):
    f.write(f"<h1 class='otitle'>P{p.id:06}: {' and '.join(cdli.get_genres(p.genre))} {cdli.get_object_type(p.object_type)}</h1>\n")
    f.write(f"<p class='otitle'>{p.period} <a href='https://cdli.ucla.edu/search/archival_view.php?ObjectID=P{p.id:06}'>CDLI</a></p>\n")
    areas_with_lines = [x for x in p.text_areas if any(x.lines)]
    for a in areas_with_lines:
        f.write(f"<section>\n")
        if len(areas_with_lines) > 1:
            f.write(f"<h1>{escape(a.name)}</h1>\n")
        show_lines = False
        if show_lines:
            f.write("<table>")
            for l in a.lines:
                f.write("<tr>")
                f.write(f"<td>{escape(l.text)}</td>\n")
                if "ml_en" in l.languages:
                    f.write(f"<td>{escape(l.languages['ml_en'])}</td>\n")
                else:
                    f.write(f"<td></td>\n")
                f.write("</tr>")
            f.write("</table>")
        else:
            texts = {p.language: " ".join(l.text for l in a.lines)}
            langs = set()
            for l in a.lines:
                for lang in l.languages:
                    if lang in language_sort:
                        langs.add(lang)
            for lang in langs:
                texts[lang] = " ".join(l.languages[lang] if lang in l.languages else " " for l in a.lines)
            langs.add(p.language)
            if "akkts" in langs and "akk" in langs:
                langs.remove("akk")
            if "suxts" in langs and "sux" in langs:
                langs.remove("sux")
            langs = sorted(list(langs), key=lambda x:language_sort[x])
            for lang in langs:
                f.write(f"<p class='lang-{lang}'><span class='langid'>{lang}</span> {escape(texts[lang])}</p>\n")
        f.write(f"</section>\n")

In [38]:
# !rm -r /Users/fak/Dropbox/Projects/CuneiformTranslators/dist

In [ ]:
def output_browser_page(path, dim_value, pubs, ignore_dim_value):
    with start_page(path + "/index", f"{len(pubs)} {escape(dim_value)}") as f:
        next_pages = output_browser(path, pubs, ignore_dim_value, f)
        for gv, gpubs in next_pages:
            output_browser_page(f"{path}/{gv}", gv, gpubs, gv)
        end_page(f)

with start_page("/404", "Not Found") as f:
    f.write(f"<p>The page you are looking for is not here.</p>\n")
    footer(f)
    
print("Writing /")
with open(get_page_file_path("/index"), "wt") as f:
    header("ML Cuneiform Translations", f)
    next_pages = output_browser("", output_pubs, "", f)
    for gv, gpubs in next_pages:
        output_browser_page(f"/{gv}", gv, gpubs, "")
    footer(f)
    
for f in glob.glob("../dist/*"):
    print(f)

Writing /404 at /Users/fak/Dropbox/Projects/CuneiformTranslators/dist/404.html
Writing /
Writing /new/index at /Users/fak/Dropbox/Projects/CuneiformTranslators/dist/new/index.html
Writing /new/sux/index at /Users/fak/Dropbox/Projects/CuneiformTranslators/dist/new/sux/index.html
Writing /new/sux/tablet/index at /Users/fak/Dropbox/Projects/CuneiformTranslators/dist/new/sux/tablet/index.html
Writing /new/sux/tablet/administrative/index at /Users/fak/Dropbox/Projects/CuneiformTranslators/dist/new/sux/tablet/administrative/index.html
Writing /new/sux/tablet/lexical/index at /Users/fak/Dropbox/Projects/CuneiformTranslators/dist/new/sux/tablet/lexical/index.html
Writing /new/sux/tablet/lexical/literary/index at /Users/fak/Dropbox/Projects/CuneiformTranslators/dist/new/sux/tablet/lexical/literary/index.html
Writing /new/sux/tablet/lexical/mathematical/index at /Users/fak/Dropbox/Projects/CuneiformTranslators/dist/new/sux/tablet/lexical/mathematical/index.html
Writing /new/sux/tablet/lexical/sc

Writing /new/sux/seal/fake/index at /Users/fak/Dropbox/Projects/CuneiformTranslators/dist/new/sux/seal/fake/index.html
Writing /new/sux/brick/index at /Users/fak/Dropbox/Projects/CuneiformTranslators/dist/new/sux/brick/index.html
Writing /new/sux/barrel/index at /Users/fak/Dropbox/Projects/CuneiformTranslators/dist/new/sux/barrel/index.html
Writing /new/sux/barrel/royal-monumental/index at /Users/fak/Dropbox/Projects/CuneiformTranslators/dist/new/sux/barrel/royal-monumental/index.html
Writing /new/sux/barrel/legal/index at /Users/fak/Dropbox/Projects/CuneiformTranslators/dist/new/sux/barrel/legal/index.html
Writing /new/sux/barrel/administrative/index at /Users/fak/Dropbox/Projects/CuneiformTranslators/dist/new/sux/barrel/administrative/index.html
Writing /new/sux/administrative/index at /Users/fak/Dropbox/Projects/CuneiformTranslators/dist/new/sux/administrative/index.html
Writing /new/sux/administrative/tablet/index at /Users/fak/Dropbox/Projects/CuneiformTranslators/dist/new/sux/adm

Writing /new/sux/royal-monumental/tablet/index at /Users/fak/Dropbox/Projects/CuneiformTranslators/dist/new/sux/royal-monumental/tablet/index.html
Writing /new/sux/royal-monumental/tablet/royal-monumental/index at /Users/fak/Dropbox/Projects/CuneiformTranslators/dist/new/sux/royal-monumental/tablet/royal-monumental/index.html
Writing /new/sux/royal-monumental/tablet/literary/index at /Users/fak/Dropbox/Projects/CuneiformTranslators/dist/new/sux/royal-monumental/tablet/literary/index.html
Writing /new/sux/royal-monumental/other-object/index at /Users/fak/Dropbox/Projects/CuneiformTranslators/dist/new/sux/royal-monumental/other-object/index.html
Writing /new/sux/royal-monumental/bulla/index at /Users/fak/Dropbox/Projects/CuneiformTranslators/dist/new/sux/royal-monumental/bulla/index.html
Writing /new/sux/royal-monumental/brick/index at /Users/fak/Dropbox/Projects/CuneiformTranslators/dist/new/sux/royal-monumental/brick/index.html
Writing /new/sux/royal-monumental/cylinder/index at /Users

Writing /new/akk/brick/index at /Users/fak/Dropbox/Projects/CuneiformTranslators/dist/new/akk/brick/index.html
Writing /new/akk/brick/royal-monumental/index at /Users/fak/Dropbox/Projects/CuneiformTranslators/dist/new/akk/brick/royal-monumental/index.html
Writing /new/akk/brick/administrative/index at /Users/fak/Dropbox/Projects/CuneiformTranslators/dist/new/akk/brick/administrative/index.html
Writing /new/akk/barrel/index at /Users/fak/Dropbox/Projects/CuneiformTranslators/dist/new/akk/barrel/index.html
Writing /new/akk/vessel/index at /Users/fak/Dropbox/Projects/CuneiformTranslators/dist/new/akk/vessel/index.html
Writing /new/akk/block/index at /Users/fak/Dropbox/Projects/CuneiformTranslators/dist/new/akk/block/index.html
Writing /new/akk/royal-monumental/index at /Users/fak/Dropbox/Projects/CuneiformTranslators/dist/new/akk/royal-monumental/index.html
Writing /new/akk/royal-monumental/other-object/index at /Users/fak/Dropbox/Projects/CuneiformTranslators/dist/new/akk/royal-monumenta

Writing /new/tablet/sux/index at /Users/fak/Dropbox/Projects/CuneiformTranslators/dist/new/tablet/sux/index.html
Writing /new/tablet/sux/administrative/index at /Users/fak/Dropbox/Projects/CuneiformTranslators/dist/new/tablet/sux/administrative/index.html
Writing /new/tablet/sux/lexical/index at /Users/fak/Dropbox/Projects/CuneiformTranslators/dist/new/tablet/sux/lexical/index.html
Writing /new/tablet/sux/lexical/literary/index at /Users/fak/Dropbox/Projects/CuneiformTranslators/dist/new/tablet/sux/lexical/literary/index.html
Writing /new/tablet/sux/lexical/mathematical/index at /Users/fak/Dropbox/Projects/CuneiformTranslators/dist/new/tablet/sux/lexical/mathematical/index.html
Writing /new/tablet/sux/lexical/school/index at /Users/fak/Dropbox/Projects/CuneiformTranslators/dist/new/tablet/sux/lexical/school/index.html
Writing /new/tablet/sux/legal/index at /Users/fak/Dropbox/Projects/CuneiformTranslators/dist/new/tablet/sux/legal/index.html
Writing /new/tablet/sux/literary/index at /Us

Writing /new/tablet/literary/index at /Users/fak/Dropbox/Projects/CuneiformTranslators/dist/new/tablet/literary/index.html
Writing /new/tablet/literary/sux/index at /Users/fak/Dropbox/Projects/CuneiformTranslators/dist/new/tablet/literary/sux/index.html
Writing /new/tablet/literary/sux/literary/index at /Users/fak/Dropbox/Projects/CuneiformTranslators/dist/new/tablet/literary/sux/literary/index.html
Writing /new/tablet/literary/sux/literary/lexical/index at /Users/fak/Dropbox/Projects/CuneiformTranslators/dist/new/tablet/literary/sux/literary/lexical/index.html
Writing /new/tablet/literary/sux/literary/mathematical/index at /Users/fak/Dropbox/Projects/CuneiformTranslators/dist/new/tablet/literary/sux/literary/mathematical/index.html
Writing /new/tablet/literary/sux/literary/royal-monumental/index at /Users/fak/Dropbox/Projects/CuneiformTranslators/dist/new/tablet/literary/sux/literary/royal-monumental/index.html
Writing /new/tablet/literary/sux/lexical/index at /Users/fak/Dropbox/Proje

Writing /new/tablet/other-genre/sux/index at /Users/fak/Dropbox/Projects/CuneiformTranslators/dist/new/tablet/other-genre/sux/index.html
Writing /new/tablet/omen/index at /Users/fak/Dropbox/Projects/CuneiformTranslators/dist/new/tablet/omen/index.html
Writing /new/tablet/scientific/index at /Users/fak/Dropbox/Projects/CuneiformTranslators/dist/new/tablet/scientific/index.html
Writing /new/tablet/scientific/akk/index at /Users/fak/Dropbox/Projects/CuneiformTranslators/dist/new/tablet/scientific/akk/index.html
Writing /new/tablet/scientific/sux/index at /Users/fak/Dropbox/Projects/CuneiformTranslators/dist/new/tablet/scientific/sux/index.html
Writing /new/tablet/ritual/index at /Users/fak/Dropbox/Projects/CuneiformTranslators/dist/new/tablet/ritual/index.html
Writing /new/tablet/ritual/sux/index at /Users/fak/Dropbox/Projects/CuneiformTranslators/dist/new/tablet/ritual/sux/index.html
Writing /new/tablet/ritual/akk/index at /Users/fak/Dropbox/Projects/CuneiformTranslators/dist/new/tablet/

Writing /new/cone/sux/royal-monumental/index at /Users/fak/Dropbox/Projects/CuneiformTranslators/dist/new/cone/sux/royal-monumental/index.html
Writing /new/cone/sux/legal/index at /Users/fak/Dropbox/Projects/CuneiformTranslators/dist/new/cone/sux/legal/index.html
Writing /new/cone/sux/administrative/index at /Users/fak/Dropbox/Projects/CuneiformTranslators/dist/new/cone/sux/administrative/index.html
Writing /new/cone/sux/other-genre/index at /Users/fak/Dropbox/Projects/CuneiformTranslators/dist/new/cone/sux/other-genre/index.html
Writing /new/cone/akk/index at /Users/fak/Dropbox/Projects/CuneiformTranslators/dist/new/cone/akk/index.html
Writing /new/cone/royal-monumental/index at /Users/fak/Dropbox/Projects/CuneiformTranslators/dist/new/cone/royal-monumental/index.html
Writing /new/cone/royal-monumental/sux/index at /Users/fak/Dropbox/Projects/CuneiformTranslators/dist/new/cone/royal-monumental/sux/index.html
